In [ ]:
import cv2
import os
from torch.utils.data import Dataset
import pickle
import random
import numpy as np
import torch

In [ ]:
class CelebADset(Dataset):
    def __init__(self, files_list):
        self.images = files_list
    
    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        return self.images[index]


In [ ]:
# CURRENTLY ASSUMES BGR AND -1 TO 1
def get_files(start=None, stop=None, augment=True, shape=(64,64)):
    files_list = []
    i = 0
    for file in os.listdir("img_align_celeba")[start: stop]:
        if not i % 1000:
            print(i)
        img = cv2.imread("img_align_celeba/" + file)
        y_start = 50
        x_start = 25
        crop_size = 128
        img = img[y_start:y_start + crop_size, x_start:x_start + crop_size]
        img = cv2.resize(img, shape)
        img = np.moveaxis(img, 2, 0)
        if augment: 
            should_flip = random.randint(0, 1) == 1
            if should_flip:
                img = np.moveaxis(img, 1, 2)
            c, h, w = img.shape
            noise = torch.FloatTensor(c, h, w).uniform_(0, 1/256)
            img = img / 256
            temp = torch.FloatTensor(img)
            img = torch.FloatTensor(img) + noise
        else:
            img = torch.FloatTensor(img)
            img = img / 255
        img = (img - 0.5) * 2
        i += 1
        files_list.append(img)
    random.shuffle(files_list)
    return files_list

In [ ]:
NUM_EXAMPLES = 202599
TRAIN_STOP = 162770
VAL_STOP = 182637

def format_celeba(shape=(64, 64), split=None, augment=True):
    if split is None:
        files = get_files(shape=shape)
    elif split == 'train':
        files = get_files(0, TRAIN_STOP, shape=shape, augment=augment)
    elif split == 'val':
        files = get_files(TRAIN_STOP, VAL_STOP, shape=shape, augment=augment)
    elif split == 'test':
        files = get_files(VAL_STOP, shape=shape, augment=augment)
    else:
        raise ValueError('unknown split')
        
    dataset = CelebADset(files)
    return dataset




In [ ]:
train_dset = format_celeba(split='train')
# val_dset = format_celeba(split='val')
test_dset = format_celeba(split='test')


In [ ]:
with open("celeba_64_bgr_-1_to_1_train.pkl", "wb") as f:
    pickle.dump(train_dset, f, protocol=2)
# with open("celeba_64_bgr_-1_to_1_val.pkl", "wb") as f:
#     pickle.dump(val_dset, f)
# with open("celeba_64_bgr_-1_to_1_test.pkl", "wb") as f:
#     pickle.dump(test_dset, f)

In [ ]:
files_list = train_dset.images
segment = len(train_dset) // 5
for i in range(5):
    with open("celeba_64_bgr_-1_to_1_train_part%i.pkl" %i, "wb") as f:
        pickle.dump(files_list[segment * i:segment * (i + 1)], f)

In [ ]:
class CelebADset(Dataset):
    def __init__(self, files_list):
        self.images = files_list
    
    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        return self.images[index]


In [1]:
import tensorflow as tf
import numpy as np
import torch.nn as nn
import torch

In [2]:
def print_param(v):
    print("Name: ", v.name)
    vector = sess.run(v)
    print("Shape: ", v.shape)
    print(vector)
    print("=" * 20)
    name = v.name.replace("/", "_")
    np.save("params/" + name, vector)

In [7]:
sess = tf.InteractiveSession()
x = np.array([[[[3., 0.],
   [6., 1.]],

#   [[0., 2.],
#    [4., 0.]],

  [[11., 30.],
   [88. ,20.]]],


 [[[0., 6.],
   [6. ,7.]],

#   [[1., 110.],
#    [2. ,20.]],

  [[0., 3.],
   [0. ,4.]]]])
print(x.shape)

tf_arr = np.moveaxis(x, 1, 3)
tf_arr = tf.convert_to_tensor(tf_arr, dtype=tf.float32)
bn = tf.contrib.layers.batch_norm(tf_arr, decay=0.9, # OK
                                    center=True, # OK
                                    scale=True, #OK
                                    epsilon=1e-5,#OK
                                    zero_debias_moving_mean=False,
                                    is_training=True)
sess.run([tf.global_variables_initializer(),
          tf.local_variables_initializer()])
bn_eval = bn.eval()
bn_comparable = np.moveaxis(bn_eval, 3, 1)
print(bn_comparable)
[print_param(v) for v in tf.trainable_variables() ]
sess.close()

#0000, 1001 --> same val = same; x__x
#1100, 1110 --> same val = same  __x_ 1000

#

(2, 2, 2, 2)
[[[[-0.22473314 -1.3034523 ]
   [ 0.85398597 -0.94387925]]

  [[-0.30661756  0.37876284]
   [ 2.4709768   0.01803633]]]


 [[[-1.3034523   0.85398597]
   [ 0.85398597  1.213559  ]]

  [[-0.7034167  -0.59519875]
   [-0.7034167  -0.55912614]]]]
Name:  BatchNorm/gamma:0
Shape:  (3,)
[1. 1. 1.]
Name:  BatchNorm/beta:0
Shape:  (3,)
[0. 0. 0.]
Name:  BatchNorm_1/gamma:0
Shape:  (2,)
[1. 1.]
Name:  BatchNorm_1/beta:0
Shape:  (2,)
[0. 0.]
Name:  BatchNorm_2/gamma:0
Shape:  (2,)
[1. 1.]
Name:  BatchNorm_2/beta:0
Shape:  (2,)
[0. 0.]
Name:  BatchNorm_3/gamma:0
Shape:  (2,)
[1. 1.]
Name:  BatchNorm_3/beta:0
Shape:  (2,)
[0. 0.]
Name:  BatchNorm_4/gamma:0
Shape:  (2,)
[1. 1.]
Name:  BatchNorm_4/beta:0
Shape:  (2,)
[0. 0.]


In [9]:
m = np.mean(x, axis=(0,2,3))
v = np.std(x, axis=(0,2,3))
print((x - m)/v)

[[[[-2.24733287e-01 -7.03416701e-01]
   [ 8.53986492e-01 -6.67344050e-01]]

  [[ 2.65185279e+00  3.78762839e-01]
   [ 3.03389938e+01  1.80363257e-02]]]


 [[[-1.30345307e+00 -4.86980793e-01]
   [ 8.53986492e-01 -4.50908142e-01]]

  [[-1.30345307e+00 -5.95198747e-01]
   [-1.30345307e+00 -5.59126096e-01]]]]


In [8]:
# PYTORCH
batchnorm = nn.BatchNorm2d(2, eps=1e-5)
# batchnorm.training=False
nn.init.constant_(batchnorm.weight, 1)
torch_arr = torch.FloatTensor(x)
torch_bn = batchnorm(torch_arr)
print(torch_bn)


tensor([[[[-0.2247, -1.3035],
          [ 0.8540, -0.9439]],

         [[-0.3066,  0.3788],
          [ 2.4710,  0.0180]]],


        [[[-1.3035,  0.8540],
          [ 0.8540,  1.2136]],

         [[-0.7034, -0.5952],
          [-0.7034, -0.5591]]]], grad_fn=<NativeBatchNormBackward>)


In [ ]:
# m = torch.mean(torch_arr, dim=(1,2,3))
# # v = torch_arr.permute(1,0,2,3)
# v = torch_arr.contiguous().view(v.size(0), -1)
# v = torch.var(v, 1)


# print(m.shape)
# print(v.shape)
# print(torch.nn.functional.batch_norm(torch_arr, m, v))


# # 0
# # 1 X
# # 2
# # 3

In [ ]:
torch.var(torch_arr, dim=1)[:,0,0]#, dim=(0,2,3))